In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

In [2]:
env = gym.make("SpaceInvaders-v0")

In [3]:
len(env.observation_space.shape)

3

In [4]:
env.action_space.n

6

In [5]:
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        return F.relu(residual + x)

In [6]:
class _dynamic_s(nn.Module):
    def __init__(self):
        super(_dynamic_s, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=3, padding=1)
        self.relu1 = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(1, 1, kernel_size=[4, 5], padding=[1, 2])
        self.relu2 = nn.LeakyReLU()
        self.res1 = ResBlock(1, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.leaky_relu(x)
        x = self.res1(x)
        x = F.leaky_relu(x)
        x = self.conv2(x)
        return x
ds = _dynamic_s()

In [7]:
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x
        x = self.bn1(self.conv1(x))
        return residual + x

In [8]:
class _dynamic_r(nn.Module):
    def __init__(self, x):
        super(_dynamic_r, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=3, stride=2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(243, 24) # there has got to be a way to fix this other than defining a helper function fuck me
        self.linear2 = nn.Linear(24, 1)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

    def _get_output_shape(self, x_dim):
        with torch.no_grad():
            return self.conv1(torch.rand(*(x_dim))).data.shape # wtf even is this function holy shit

dr = _dynamic_r((1, 1, 192, 168))
np.prod(list(dr._get_output_shape((1, 1, 192, 168))))

7885

In [9]:
def get_output_shape(model, tensor_dim):
    with torch.no_grad():
        return model(torch.rand(*(tensor_dim))).data.shape

In [12]:
obs = torch.tensor(env.reset(), dtype=torch.float32).unsqueeze(0)
obs.transpose_(1, 3).transpose_(2, 3)

_representation = nn.Sequential(
    nn.Conv2d(3, 1, kernel_size=6),
    nn.LeakyReLU(),
    nn.MaxPool2d(5),
    nn.Conv2d(1, 1, kernel_size=4)
)

obs = _representation(obs)

# This bottom code is just cursed, idk how to do this better since obs has a dimension of 4
# action_one_hot = torch.zeros([1, 1, 1, obs.shape[-1]], dtype=torch.float32)
# index = torch.tensor([[[[2]]]])
# action_one_hot.scatter_(3, index, 1.0)
# x = torch.cat((obs, action_one_hot), dim=2)

# x = ds(x)
# r = dr(x)
# r.item()

In [14]:
_prediction_p = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=3, stride=2),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(234, 24), # same goes for here
    nn.Linear(24, env.action_space.n),
)

_prediction_v = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=3, stride=2),
    nn.Flatten(),
    nn.Linear(234, 24), # same goes for here
    nn.Linear(24, 1),
)

p = _prediction_p(x)
v = _prediction_v(x)

# policy = {a: math.exp(p[a].item()) for a in range(env.action_space.n)}
# policy
# p.squeeze_(0).detach().numpy()

In [43]:
list(get_output_shape(_representation, (1, 3, 210, 160)))

[1, 1, 38, 28]